**Title**: Update Acquisition container timestamp (shown in UI) from Dicom Tag SeriesTime in Dicom file 

**Date**:  04-03-2020  

**Description**:  
This notebook can be used to update the Acquisition container timestamp (shown in UI) is not correct.  

**Functions**: 

    1.   `get_updated_timestamp` : Get the updated timestamp
    2.   `modify_time` : Comparing the timestamp shows on the Acquisition container and SeriesTime in Dicom file
    3.  `update_container_timestamp` : Update the acquisition timestamp 






# Install and import dependencies

In [0]:
# Install specific packages required for this notebook
!pip install flywheel-sdk pandas

In [0]:
# Import packages
from getpass import getpass
import logging
import os
import datetime

import pandas as pd
import flywheel

In [0]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

# Flywheel API Key and Client

In [0]:
API_KEY = getpass('Enter API_KEY here: ')

Instantiate the Flywheel API client

In [0]:
fw = flywheel.Client(API_KEY or os.environ.get('FW_KEY'))

Show Flywheel logging information

In [0]:
log.info('You are now logged in as %s to %s', fw.get_current_user()['email'], fw.get_config()['site']['api_url'])

# Constants

In [0]:

PROJECT_LABEL = input('Enter your project label here: ')

project = fw.projects.find_first(f'label={PROJECT_LABEL}')


# Helper functions

In [0]:
def update_acquisition_timestamp(item):
  """Get and modify the updated timestamp., if there is any discrepancy between the two timestamps.

  Args:
        item (dict): Dictionary with the following required keys: 'timestamp', 'series_time', 'acq_id'
        
  Return:

        (datetime): Updated Timestamp
        
  """

   correctTmstmp = modify_time(item['timestamp'], item['series_time'])
   #only update the container if there is any changes
   if correctTmstmp != item['timestamp']:
         update_container_timestamp(item['acq_id'], correctTmstmp)  
   return correctTmstmp


def modify_time(timestamp, series_time):
  """Modify the timestamp.

  Args:
        timestamp (datetime): Timestamp 
        series_time (datetime): Reference Timestamp

  Return:

        (datetime): Updated Timestamp
        
  """

      if (timestamp.hour != series_time.hour) or (timestamp.minute != series_time.minute) or (timestamp.second != series_time.second):
        timestamp = timestamp.replace(hour=series_time.hour, minute=series_time.minute, second=series_time.second)
      return timestamp


def update_container_timestamp(acq_id, ts):
   """Update the acquisition container timestamp with `ts`. 

  Args:
        acq_id (str) : Acquisition ID
        ts (datetime) : Timestamp
        
  """
  acq = fw.get_acquisition(acq_id)
  acq.update({'timestamp': ts})

# Main script

In [0]:
df = pd.DataFrame()

for session in project.sessions.iter():
    for acquisition in session.acquisitions.iter():
      #reload is necessary to load the entire acquisition
      acquisition = acquisition.reload()
      for file in acquisition.files : 
        if file['type'] == 'dicom':
          item = {
            'series_number': file.info.get('SeriesNumber'),
            'acq_label': acquisition.label,
            'acq_id': acquisition.id, 
            'timestamp': acquisition.timestamp,
            'series_time': datetime.datetime.strptime(file.info.get('SeriesTime'), '%H%M%S.%f'), 
          }    
          item['updated_timestamp'] = get_updated_timestamp(item)
            
          # append to the table
          df = df.append(item, ignore_index=True)


Here, we will get the acquistion container timestamp (`timestamp`) and the series timestamp (`series_time`) generated from the DICOM file. The `get_updated_timestamp` function will be call and return with the updated_timestamp.

## Display Updated Acquisition Info


In [0]:
# display all of the columns 
pd.options.display.max_columns = None

display(df.head(50))